# PackLLM – MMLU Fusion Experiment (Google Colab)
Runs the *opt-fusion* experiment on the MMLU benchmark using 
`NousResearch/Llama-2-7b-hf` and three additional 7-B models.
Results are stored in `outputs/` and key metrics printed at the end.

In [ ]:
# ↳ Install libraries (quiet mode)
!pip -q install transformers==4.41.1 accelerate datasets evaluate scikit-learn sentencepiece

In [ ]:
# ↳ Clone the PackLLM repo (public, no auth needed)
!git clone -q https://github.com/tenet-diver/ppack_of_llamas.git
%cd ppack_of_llamas

In [ ]:
# ↳ Install any repo-specific requirements
!pip -q install -r downstream_tasks/requirements.txt || true

In [ ]:
# ↳ Run the fusion experiment for the MMLU benchmark
import os, json, pathlib, subprocess, textwrap
MODEL_NAMES = ','.join([
    'NousResearch/Llama-2-7b-hf',
    'mistralai/Mistral-7B-v0.1',
    'microsoft/phi-2',
    'Deci/DeciLM-7B'
])
cmd = textwrap.dedent(f'''\
    python downstream_tasks/main.py \n      --task_name mmlu \n      --fusion opt \n      --model_name "{MODEL_NAMES}" \n      --few_shot 0 \n      --data_cache_dir datasets \n      --output_dir outputs/fusion_opt_mmlu \n      --annotation_size 5 \n      --seed 1\
    ''')
print('Executing:
', cmd)
subprocess.run(cmd, shell=True, check=True)

In [ ]:
# ↳ Display the summary results if the script produced a metrics file
import glob, pandas as pd, json, os, pprint, pathlib
results_files = glob.glob('outputs/fusion_opt_mmlu/**/*.json', recursive=True)
if results_files:
    df_list = []
    for f in results_files:
        with open(f) as fh:
            data = json.load(fh)
        df_list.append({'file': pathlib.Path(f).name, **data})
    display(pd.DataFrame(df_list))
else:
    print('No JSON results found yet. Check logs above.')